# Active harmonic oscillator in a viscous liquid with thermal motion

In [3]:
import numpy as np
import matplotlib.pyplot as plt

class HarmonicOscillatorLangevin:
    def __init__(self, k=1.0, m=1.0, gamma=0.1, T=300.0, x0=1.0, v0=0.0):
        """Initialize the Langevin oscillator with given parameters."""
        self.k = k
        self.m = m
        self.gamma = gamma
        self.T = T
        self.k_B = 1 #1.380649e-23  
        self.x = x0
        self.v = v0
        self.time_evolution_strategy = None

    def set_time_evolution_strategy(self, strategy):
        """Set the strategy for time evolution."""
        self.time_evolution_strategy = strategy

    def compute_force(self):
        """Compute the deterministic part of the force for the Langevin oscillator."""
        return -self.k * self.x

    def time_evolution(self, dt, num_steps):
        """Simulate the time evolution using the specified strategy for a given number of steps."""
        return self.time_evolution_strategy.evolve(self, dt, num_steps)
    
    def plot_phase_space_trajectory(self, x_values, v_values):
        """Plot the phase space trajectory of the Langevin oscillator."""
        plt.figure(figsize=(8, 6))
        plt.plot(x_values, v_values, label='Phase Space Trajectory')
        plt.xlabel('Position x')
        plt.ylabel('Velocity v')
        plt.title('Phase Space Trajectory using Langevin Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    def plot_position_vs_time(self, times, x_values):
        """Plot the position as a function of time for the Langevin oscillator."""
        plt.figure(figsize=(8, 6))
        plt.plot(times, x_values, label='Position vs Time')
        plt.xlabel('Time')
        plt.ylabel('Position x')
        plt.title('Position using Langevin Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()

    def plot_velocity_vs_time(self, times, v_values):
        """Plot the velocity as a function of time for the Langevin oscillator."""
        plt.figure(figsize=(8, 6))
        plt.plot(times, v_values, label='Velocity vs Time')
        plt.xlabel('Time')
        plt.ylabel('Velocity v')
        plt.title('Velocity using Langevin Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()


In [1]:
###
# time evolution
###
class EulerMethod:
    @staticmethod
    def evolve(oscillator, dt, num_steps):
        times = np.linspace(0, dt*num_steps, num_steps+1)
        x_values, v_values = [oscillator.x], [oscillator.v]
        
        for _ in range(num_steps):
            oscillator.v += oscillator.compute_force() / oscillator.m * dt - oscillator.gamma * oscillator.v * dt
            oscillator.x += oscillator.v * dt
            
            x_values.append(oscillator.x)
            v_values.append(oscillator.v)
        
        return times, x_values, v_values


class EulerMaruyamaMethod:
    @staticmethod
    def evolve(oscillator, dt, num_steps):
        times = np.linspace(0, dt*num_steps, num_steps+1)
        x_values, v_values = [oscillator.x], [oscillator.v]
        
        for _ in range(num_steps):
            deterministic_force = oscillator.compute_force()
            stochastic_force = np.sqrt(2. * oscillator.gamma * oscillator.k_B * oscillator.T) * np.random.normal()
            
            oscillator.v += (deterministic_force / oscillator.m - oscillator.gamma * oscillator.v) * dt + stochastic_force / oscillator.m * np.sqrt(dt)
            oscillator.x += oscillator.v * dt
            
            x_values.append(oscillator.x)
            v_values.append(oscillator.v)
        
        return times, x_values, v_values


class LangevinVelocityVerletMethod:
    @staticmethod
    def evolve(oscillator, dt, num_steps):
        times = np.linspace(0, dt*num_steps, num_steps+1)
        x_values, v_values = [oscillator.x], [oscillator.v]
        
        sqrt_dt = np.sqrt(2 * oscillator.gamma * oscillator.k_B * oscillator.T / oscillator.m) * np.sqrt(dt)
        
        for _ in range(num_steps):
            # Half update of velocity with deterministic and stochastic forces
            a = oscillator.compute_force() / oscillator.m
            v_half = oscillator.v + 0.5 * dt * (a - oscillator.gamma * oscillator.v) + sqrt_dt * np.random.normal(0, 1)
            
            # Update position
            oscillator.x += dt * v_half

            # New acceleration
            a_new = oscillator.compute_force() / oscillator.m

            # Complete velocity update
            oscillator.v = v_half + 0.5 * dt * (a_new - oscillator.gamma * oscillator.v) 

            x_values.append(oscillator.x)
            v_values.append(oscillator.v)
        
        return times, x_values, v_values
    


# Add activity that is produced at the level of individual particle

In [2]:
class ActiveHarmonicOscillatorLangevin(HarmonicOscillatorLangevin):
    def __init__(self, F_active=0.1, switch_steps=100, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.F_active = F_active
        self.switch_steps = switch_steps
        self.steps_since_last_switch = 0
        self.active_direction = np.sign(np.random.randn())  # Randomly set to +1 or -1
        
    def compute_force(self):
        """Compute the total force including the harmonic, damping, and active terms."""
        harmonic_force = super().compute_force()
        active_force = # EDIT HERE
        
        # Check if we need to switch the direction of the active force
        self.steps_since_last_switch += 1
        if self.steps_since_last_switch >= self.switch_steps:
            self.active_direction = np.sign(np.random.randn())
            self.steps_since_last_switch = 0
        
        return harmonic_force + active_force


SyntaxError: invalid syntax (401400913.py, line 12)

# Let's test our active oscillator: Passive vs. Active


In [4]:

###
# Passive
###
# Create an instance and set the evolution strategy
active_oscillator = ActiveHarmonicOscillatorLangevin(gamma=1.0, T=0.1, F_active=0., switch_steps=1000)

active_oscillator.set_time_evolution_strategy(LangevinVelocityVerletMethod)
times, x_values, v_values = active_oscillator.time_evolution(dt=0.01, num_steps=10000)

# Plotting results
active_oscillator.plot_phase_space_trajectory(x_values, v_values)
active_oscillator.plot_position_vs_time(times, x_values)

###
# Active
###
# Create an instance and set the evolution strategy
active_oscillator = ActiveHarmonicOscillatorLangevin(gamma=1.0, T=0.1, F_active=5., switch_steps=1000)

active_oscillator.set_time_evolution_strategy(LangevinVelocityVerletMethod)
times, x_values, v_values = active_oscillator.time_evolution(dt=0.01, num_steps=10000)

# Plotting results
active_oscillator.plot_phase_space_trajectory(x_values, v_values)
active_oscillator.plot_position_vs_time(times, x_values)

NameError: name 'ActiveHarmonicOscillatorLangevin' is not defined